In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import itertools
import sys
sys.path.append('..')
from security_game.target import Target
from security_game.green_security_game import GreenSecurityGame
from security_game.infra_security_game import InfraSecurityGame

from solvers.mip import mip
from solvers.nash import nash
from solvers.double_oracle import double_oracle
from solvers.double_oracle_sf import double_oracle_sf
from solvers.no_regret import regret_matching

from utils.random_utils import generate_random_utility_matrix_like, generate_random_target_utility_matrix_like, generate_random_schedule_mapping_like

import time
from matplotlib import pyplot as plt
from collections import defaultdict, Counter

In [2]:
seeds = [1,2,3,4,5,6,7,8,9,10]

# GSG

## NFG

In [3]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]

schedule_form_kwargs = {
    "schedule_form": False,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

In [4]:
gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=False, 
             general_sum=False, 
             **schedule_form_kwargs)

In [5]:
gsg.utility_matrix.shape

(9075, 11)

In [6]:
nruntimes = []
nsupports = []
nus = []
mip_us_per_seed = []
mip_supports_per_seed = []
mip_runtimes_per_seed = []

for seed in seeds:
    print(f"starting seed {seed}")
    random_matrix = generate_random_utility_matrix_like(gsg.utility_matrix)
    start = time.time()
    nD_a, nD_d, nu = nash(random_matrix)
    end = time.time()
    nruntime = end-start
    nsupport = sum([1 for p in nD_d if p!=0])
    nus.append(nu)
    print(nsupport,nu)
    nruntimes.append(nruntime)
    nsupports.append(nsupport)
    
    mip_us = []
    mip_supports = []
    mip_runtimes = []
    
    for i in range(1,60):
        start = time.time()
        print(f"starting i={i} at time {start}")
        mu, mD_d = mip(random_matrix,i)
        end = time.time()
        print(f"finished i={i} in {end-start} seconds with u={mu}")
        msupport = sum([1 for p in mD_d if p!=0])
        mip_us.append(mu)
        mip_supports.append(i)
        mip_runtimes.append(end-start)
        if abs(mu-nu) <= 1e-12:
            mip_us_per_seed.append(mip_us)
            mip_supports_per_seed.append(mip_supports)
            mip_runtimes_per_seed.append(mip_runtimes)
            break

starting seed 1
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2455389
Academic license 2455389 - for non-commercial use only - registered to jc___@columbia.edu
11 -0.2289783193897651
starting i=1 at time 1746175071.5342908
finished i=1 in 16.936410427093506 seconds with u=-0.5044326170711041
starting i=2 at time 1746175088.4707012
finished i=2 in 33.8625864982605 seconds with u=-0.31089369397920674
starting i=3 at time 1746175122.3343465
finished i=3 in 75.9526424407959 seconds with u=-0.26797197513330717
starting i=4 at time 1746175198.286989
finished i=4 in 30.822558641433716 seconds with u=-0.2539575820322724
starting i=5 at time 1746175229.1105487
finished i=5 in 7.501953601837158 seconds with u=-0.24143695009263985
starting i=6 at time 1746175236.6125023
finished i=6 in 8.403522729873657 seconds with u=-0.23710604221520137
starting i=7 at time 1746175245.016025
finished i=7 in 5.251911878585815 seconds with u=-0.23296215805545575
starting i=8 a

In [8]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps":[8 for i in range(len(seeds))],
    "num_attackers":[1 for i in range(len(seeds))],
    "num_defenders":[1 for i in range(len(seeds))],
    "num_clusters":[10 for i in range(len(seeds))],
    "dims":[7 for i in range(len(seeds))],
    "defense_time_threshold":[1 for i in range(len(seeds))],
    "force_return":[False for i in range(len(seeds))],
    "num_defender_actions": [len(gsg.defender_actions) for i in range(len(seeds))],
    "nash_value":nus,
    "nash_support":nsupports,
    "nash_runtime":nruntimes,
    "mip_value":mip_us_per_seed,
    "mip_support":mip_supports_per_seed,
    "mip_runtime":mip_runtimes_per_seed,
})
df


,seed,num_timesteps,num_attackers,num_defenders,num_clusters,dims,defense_time_threshold,force_return,num_defender_actions,nash_value,nash_support,nash_runtime,mip_value,mip_support,mip_runtime
0,1,8,1,1,10,7,1,False,9075,-0.228978,11,0.904140,"[-0.5044326170711041, -0.31089369397920674, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[16.936410427093506, 33.8625864982605, 75.9526..."
1,2,8,1,1,10,7,1,False,9075,-0.220226,11,1.524903,"[-0.4182092178640903, -0.27913876557878464, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[8.704082727432251, 95.46329140663147, 67.6560..."
2,3,8,1,1,10,7,1,False,9075,-0.226528,11,1.433374,"[-0.45549993364587926, -0.2986982951342234, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[4.228636741638184, 262.91377210617065, 41.463..."
3,4,8,1,1,10,7,1,False,9075,-0.212780,11,0.989177,"[-0.40808381310089725, -0.28319613705544316, -...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[3.7767703533172607, 80.34049606323242, 54.604..."
4,5,8,1,1,10,7,1,False,9075,-0.254547,11,0.755342,"[-0.4665794904475682, -0.31029687693367736, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[13.480432748794556, 277.72857213020325, 69.92..."
5,6,8,1,1,10,7,1,False,9075,-0.226330,11,1.140028,"[-0.4586682047678752, -0.3078526169046087, -0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[3.394660711288452, 60.965864181518555, 108.82..."
6,7,8,1,1,10,7,1,False,9075,-0.229392,11,0.827885,"[-0.4840402518943999, -0.3310506919707057, -0....","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[14.644285440444946, 72.74372982978821, 52.324..."
7,8,8,1,1,10,7,1,False,9075,-0.241549,11,1.370214,"[-0.4460545198711925, -0.30883189277340395, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[9.689504384994507, 108.74848747253418, 41.394..."
8,9,8,1,1,10,7,1,False,9075,-0.231951,11,0.854400,"[-0.419637369493265, -0.3149184911583927, -0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[3.568195343017578, 73.07053542137146, 49.8798..."
9,10,8,1,1,10,7,1,False,9075,-0.229369,11,0.921066,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."


In [9]:
df.to_csv("GSG_NFG_T8_1D1A_SPARSITY_RANDOM_MATRIX.csv")

### T=9

In [ ]:
gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=1, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=9, 
             generate_utility_matrix=True, 
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=False, 
             general_sum=False, 
             **schedule_form_kwargs)

In [ ]:
gsg.utility_matrix.shape

In [7]:
nruntimes9 = []
nsupports9 = []
nus9 = []
mip_us_per_seed9 = []
mip_supports_per_seed9 = []
mip_runtimes_per_seed9 = []

for seed in seeds:
    print(f"starting seed {seed}")
    random_matrix = generate_random_utility_matrix_like(gsg.utility_matrix)
    start = time.time()
    nD_a, nD_d, nu = nash(random_matrix)
    end = time.time()
    nruntime = end-start
    nsupport = sum([1 for p in nD_d if p!=0])
    nus9.append(nu)
    print(nsupport,nu)
    nruntimes9.append(nruntime)
    nsupports9.append(nsupport)
    
    mip_us9 = []
    mip_supports9 = []
    mip_runtimes9 = []
    
    for i in range(1,60):
        start = time.time()
        print(f"starting i={i} at time {start}")
        mu, mD_d = mip(random_matrix,i)
        end = time.time()
        print(f"finished i={i} in {end-start} seconds with u={mu}")
        msupport = sum([1 for p in mD_d if p!=0])
        mip_us9.append(mu)
        mip_supports9.append(i)
        mip_runtimes9.append(end-start)
        if abs(mu-nu) <= 1e-12:
            mip_us_per_seed9.append(mip_us)
            mip_supports_per_seed9.append(mip_supports)
            mip_runtimes_per_seed9.append(mip_runtimes)
            break

starting seed 1
11 -0.24128442453428955
starting i=1 at time 1746177828.6424155
finished i=1 in 6.852816581726074 seconds with u=-0.4165350262255697
starting i=2 at time 1746177835.4962506
finished i=2 in 157.48959922790527 seconds with u=-0.3185378806114157
starting i=3 at time 1746177992.9858499
finished i=3 in 46.539321422576904 seconds with u=-0.2770683451533938
starting i=4 at time 1746178039.5251713
finished i=4 in 69.29173541069031 seconds with u=-0.2643951439313114
starting i=5 at time 1746178108.8179088
finished i=5 in 25.693912267684937 seconds with u=-0.2541441982834858
starting i=6 at time 1746178134.5128314
finished i=6 in 17.616273641586304 seconds with u=-0.24842366538158642
starting i=7 at time 1746178152.129105
finished i=7 in 12.283194780349731 seconds with u=-0.24605841767782416
starting i=8 at time 1746178164.4133866
finished i=8 in 9.21664834022522 seconds with u=-0.24379585813985077
starting i=9 at time 1746178173.630035
finished i=9 in 1.7898154258728027 seconds 

In [10]:
df = pd.DataFrame({
    "seed":seeds,
    "num_timesteps":[9 for i in range(len(seeds))],
    "num_attackers":[1 for i in range(len(seeds))],
    "num_defenders":[1 for i in range(len(seeds))],
    "num_clusters":[10 for i in range(len(seeds))],
    "dims":[7 for i in range(len(seeds))],
    "defense_time_threshold":[1 for i in range(len(seeds))],
    "force_return":[False for i in range(len(seeds))],
    "num_defender_actions": [len(gsg.defender_actions) for i in range(len(seeds))],
    "nash_value":nus9,
    "nash_support":nsupports9,
    "nash_runtime":nruntimes9,
    "mip_value":mip_us_per_seed9,
    "mip_support":mip_supports_per_seed9,
    "mip_runtime":mip_runtimes_per_seed9,
})
df


,seed,num_timesteps,num_attackers,num_defenders,num_clusters,dims,defense_time_threshold,force_return,num_defender_actions,nash_value,nash_support,nash_runtime,mip_value,mip_support,mip_runtime
0,1,9,1,1,10,7,1,False,9075,-0.241284,11,0.974999,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."
1,2,9,1,1,10,7,1,False,9075,-0.238688,11,0.883014,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."
2,3,9,1,1,10,7,1,False,9075,-0.223516,11,0.829408,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."
3,4,9,1,1,10,7,1,False,9075,-0.190272,10,0.902944,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."
4,5,9,1,1,10,7,1,False,9075,-0.224730,11,0.834738,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."
5,6,9,1,1,10,7,1,False,9075,-0.228936,11,1.098604,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."
6,7,9,1,1,10,7,1,False,9075,-0.202619,11,1.204323,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."
7,8,9,1,1,10,7,1,False,9075,-0.237784,11,1.442981,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."
8,9,9,1,1,10,7,1,False,9075,-0.224464,11,1.048429,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."
9,10,9,1,1,10,7,1,False,9075,-0.237946,11,0.991784,"[-0.3517097708107143, -0.27549143638279683, -0...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[2.254662036895752, 43.00606632232666, 24.2394..."


In [11]:
df.to_csv("GSG_NFG_T9_1D1A_SPARSITY_RANDOM_MATRIX.csv")

## SF

In [3]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

In [4]:
gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=8, num_columns=8)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             randomize_target_utility_matrix=True,
             defense_time_threshold=2, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

In [5]:
gsg.schedule_form_dict["defender_utility_matrix"].shape

(361, 10)

In [8]:
gsg.schedule_form_dict["target_utilities"]

array([[-0.62753162, -0.52923505, -0.55387137, -0.09472521, -0.19384162,
        -0.61657655, -0.57166475, -0.26644237, -0.38211525, -0.39464848],
       [-0.62440902, -0.3886818 , -0.5013188 , -0.03631222, -0.22627576,
        -0.28217694, -0.65916686, -0.34504137, -0.04280739, -0.54048057],
       [ 0.62440902,  0.3886818 ,  0.5013188 ,  0.03631222,  0.22627576,
         0.28217694,  0.65916686,  0.34504137,  0.04280739,  0.54048057],
       [ 0.62753162,  0.52923505,  0.55387137,  0.09472521,  0.19384162,
         0.61657655,  0.57166475,  0.26644237,  0.38211525,  0.39464848]])

In [6]:
start = time.time()
nD_a, nD_d, nu = nash(gsg.schedule_form_dict["defender_utility_matrix"])
end = time.time()
nruntime = end-start
nsupport = sum([1 for p in nD_d if p!=0])
nsupport,nu

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2455389
Academic license 2455389 - for non-commercial use only - registered to jc___@columbia.edu


(1, -0.9472700553028538)

In [290]:
mip_us = []
mip_supports = []
mip_runtimes = []

for i in range(1,10):
    start = time.time()
    print(f"starting i={i} at time {start}")
    mu, mD_d = mip(gsg.schedule_form_dict["defender_utility_matrix"],i)
    end = time.time()
    print(f"finished i={i} in {end-start} seconds with u={mu}")
    msupport = sum([1 for p in mD_d if p!=0])
    mip_us.append(mu)
    mip_supports.append(i)
    mip_runtimes.append(end-start)
    if abs(mu-nu) <= 1e-12:
        break

starting i=1 at time 1745986556.0693774
finished i=1 in 0.09237241744995117 seconds with u=-0.5226130653266332
starting i=2 at time 1745986556.1617498
finished i=2 in 0.24329137802124023 seconds with u=-0.3366140189826578
starting i=3 at time 1745986556.4050412
finished i=3 in 0.13461565971374512 seconds with u=-0.28245611317941893
starting i=4 at time 1745986556.540661
finished i=4 in 0.3160886764526367 seconds with u=-0.28007355003955064
starting i=5 at time 1745986556.8567498
finished i=5 in 0.07416820526123047 seconds with u=-0.2772988260969091


In [291]:
df = pd.DataFrame({
    "num_timesteps":[8 for i in range(len(mip_supports))],
    "num_attackers":[1 for i in range(len(mip_supports))],
    "num_defenders":[2 for i in range(len(mip_supports))],
    "num_clusters":[10 for i in range(len(mip_supports))],
    "dims":[10 for i in range(len(mip_supports))],
    "defense_time_threshold":[2 for i in range(len(mip_supports))],
    "force_return":[True for i in range(len(mip_supports))],
    "num_defender_actions": [gsg.schedule_form_dict["defender_utility_matrix"].shape[0] for i in range(len(mip_supports))],
    "nash_value":[nu for i in range(len(mip_supports))],
    "nash_support":[nsupport for i in range(len(mip_supports))],
    "nash_runtime":[nruntime for i in range(len(mip_supports))],
    "mip_value":mip_us,
    "mip_support":mip_supports,
    "mip_runtime":mip_runtimes,
})
df

,num_timesteps,num_attackers,num_defenders,num_clusters,dims,defense_time_threshold,force_return,num_defender_actions,nash_value,nash_support,nash_runtime,mip_value,mip_support,mip_runtime
0,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.522613,1,0.092372
1,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.336614,2,0.243291
2,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.282456,3,0.134616
3,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.280074,4,0.316089
4,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.277299,5,0.074168


In [292]:
df.to_csv("GSG_SF_T8_SPARSITY.csv")

# ISG

In [2]:
gdf = gpd.read_file("chinatown_infra.geojson")

# Step 1: Handle relevant columns
infra_columns = [
    "id", "name", "power", "man_made", "amenity",
    "generator:method", "generator:source", "geometry"
]
available_columns = [col for col in infra_columns if col in gdf.columns]
gdf = gdf[available_columns].copy()

# Step 2: Extract generator type if present
gdf["generator_type"] = gdf.get("generator:method")
if "generator_type" not in gdf.columns or gdf["generator_type"].isnull().all():
    gdf["generator_type"] = gdf.get("generator:source")

# Step 3: Construct unified 'type' column
gdf["type"] = gdf.get("power")
if "amenity" in gdf.columns:
    gdf["type"] = gdf["type"].combine_first(gdf["amenity"])
if "man_made" in gdf.columns:
    gdf["type"] = gdf["type"].combine_first(gdf["man_made"])

# Step 4: Refine generator classification (solar vs. other)
gdf.loc[(gdf["type"] == "generator") & (gdf["generator_type"] == "photovoltaic"), "type"] = "solar_generator"
gdf.loc[(gdf["type"] == "generator") & (gdf["generator_type"] == "solar"), "type"] = "solar_generator"

# Step 5: Drop raw columns now that 'type' is finalized
df_simple = gdf[["id", "name", "type", "geometry"]].copy()

# Step 6: Separate nodes and ways
df_nodes = df_simple[df_simple["id"].str.contains("node")].copy()
df_nodes["x"] = df_nodes.geometry.x
df_nodes["y"] = df_nodes.geometry.y
df_nodes = df_nodes.drop(columns=["geometry"])

df_ways = df_simple[df_simple["id"].str.contains("way")].copy()
df_ways = df_ways.set_geometry("geometry").to_crs("EPSG:32618")
df_ways["centroid"] = df_ways.geometry.centroid
df_ways = df_ways.set_geometry("centroid").to_crs("EPSG:4326")
df_ways["x"] = df_ways.geometry.x
df_ways["y"] = df_ways.geometry.y
df_ways = df_ways.drop(columns=["geometry", "centroid"])

# Step 7: Combine nodes and ways
df_combined = pd.concat([df_nodes, df_ways], ignore_index=True)
df_combined = pd.concat([df_nodes, df_ways], ignore_index=True)
ny_blocks_gdf =  gpd.read_file("tl_2020_36_tabblock20.shp")
INFRA_WEIGHTS = {
    # Power Infrastructure
    "plant": 1.5,
    "generator": 1.35,
    "solar_generator": 0.95,
    "substation": 1.45,
    "transformer": 1.25,
    "tower": 1.1,
    "pole": 0.85,
    "line": 1.0,
    "minor_line": 0.9,
    "cable": 0.95,
    "switchgear": 1.2,
    "busbar": 0.8,
    "bay": 0.85,
    "converter": 1.05,
    "insulator": 0.75,
    "portal": 0.75,
    "connection": 0.7,
    "compensator": 1.0,
    "rectifier": 0.95,
    "inverter": 0.95,
    "storage": 0.9,

    # Healthcare
    "hospital": 1.5,
    "clinic": 1.35,

    # Education
    "school": 1.25,
    "university": 1.4,

    # Water & Sanitation
    "water_works": 1.45,
    "wastewater_plant": 1.4,

    # Government & Emergency Services
    "fire_station": 1.3,
    "police": 1.4,
    "courthouse": 1.2,

    # Critical Infrastructure
    "bunker_silo": 1.0,

    # Communications
    "communications_tower": 1.25,
}

## SF

In [3]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 3,
    "defender_penalty_factor": 3,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

# Bounding box for Hoboken, NJ
# bbox_hoboken_small = (40.752635, 40.745600, -74.030386,-74.043903)
bbox_hoboken_low = (40.745411, 40.735486, -74.025857,-74.041479)
bbox_hoboken_east = (40.748337, 40.734641,-74.022961,-74.031286)
bbox_downtown = (40.718721, 40.714078, -73.996074, -74.002651)
bbox_downtown_large = (40.7215, 40.710, -73.9935, -74.010)
# 40.7060, -74.0140, 40.7205, -73.9935
college_police = (40.743293077312465, -74.02670221027175)
police_station = (40.73768931976651, -74.02990519431108)
traffic_police = (40.7366602084371, -74.03449866349136)
downtown_station = (40.71232433042349, -74.00187755238431)
fifth_ave_station = (40.71637413934789, -73.9973285259067)
fifth_precinct = (40.71625547686622, -73.99736909131171)
booking_station = (40.716191530904815, -74.00102237385177)
police_plaza = (40.71236124409745, -74.00173715463521)
troop_nyc = (40.71657885026091, -74.00641139014367)
first_precinct = (40.720411300417446, -74.0070247584372)

In [63]:
isg = InfraSecurityGame(df_combined, ny_blocks_gdf, INFRA_WEIGHTS, bbox=bbox_downtown_large)
isg.generate(num_attackers=1, 
             num_defenders=3, 
             home_base_assignments=[(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza),(fifth_precinct,booking_station, troop_nyc, first_precinct, police_plaza)],
             num_timesteps=9, 
             generate_utility_matrix=True, 
             generate_actions=False, 
             force_return=True, 
             defense_time_threshold=3, 
             **general_sum_kwargs, 
             **schedule_form_kwargs)

#10 timesteps, 3 def, dt 2: (11, -0.4043878750324344)

In [64]:
isg.schedule_form_dict["defender_utility_matrix"].shape

(29791, 23)

In [30]:
#9 timesteps 3 def 3dt
start = time.time()
nD_a, nD_d, nu = nash(isg.schedule_form_dict["defender_utility_matrix"])
end = time.time()
nruntime = end-start
nsupport = sum([1 for p in nD_d if p!=0])
nsupport,nu

(11, -0.420864964164455)

In [33]:
#9 timesteps 3 def 2dt
start = time.time()
nD_a, nD_d, nu = nash(isg.schedule_form_dict["defender_utility_matrix"])
end = time.time()
nruntime = end-start
nsupport = sum([1 for p in nD_d if p!=0])
nsupport,nu

(9, -0.4176016505451655)

In [62]:
start = time.time()
nD_a, nD_d, nu = nash(isg.schedule_form_dict["defender_utility_matrix"])
end = time.time()
nruntime = end-start
nsupport = sum([1 for p in nD_d if p!=0])
nsupport,nu

(11, -0.420864964164455)

In [ ]:
mip_us = []
mip_supports = []
mip_runtimes = []

for i in range(1,10):
    start = time.time()
    print(f"starting i={i} at time {start}")
    mu, mD_d = mip(isg.schedule_form_dict["defender_utility_matrix"],i)
    end = time.time()
    print(f"finished i={i} in {end-start} seconds with u={mu}")
    msupport = sum([1 for p in mD_d if p!=0])
    mip_us.append(mu)
    mip_supports.append(i)
    mip_runtimes.append(end-start)
    if abs(mu-nu) <= 1e-12:
        break

starting i=1 at time 1745993149.0215442
finished i=1 in 39.68994855880737 seconds with u=-0.49543616569765425
starting i=2 at time 1745993188.7185214
finished i=2 in 185.47285151481628 seconds with u=-0.4518894049262475
starting i=3 at time 1745993374.1913729
finished i=3 in 43.33791470527649 seconds with u=-0.44334275147120084
starting i=4 at time 1745993417.5292876
finished i=4 in 67.4903175830841 seconds with u=-0.42472217747854063
starting i=5 at time 1745993485.0196052
finished i=5 in 926.8983609676361 seconds with u=-0.42302295804969314
starting i=6 at time 1745994411.9179661
finished i=6 in 2541.8093144893646 seconds with u=-0.4213980041933654
starting i=7 at time 1745996953.7272806


In [52]:
df = pd.DataFrame({
    "num_timesteps":[9 for i in range(len(mip_supports))],
    "num_attackers":[2 for i in range(len(mip_supports))],
    "num_defenders":[1 for i in range(len(mip_supports))],
    "num_clusters":[10 for i in range(len(mip_supports))],
    "dims":[7 for i in range(len(mip_supports))],
    "defense_time_threshold":[1 for i in range(len(mip_supports))],
    "force_return":[False for i in range(len(mip_supports))],
    "num_defender_actions": [len(gsg.defender_actions) for i in range(len(mip_supports))],
    "nash_value":[nu for i in range(len(mip_supports))],
    "nash_support":[nsupport for i in range(len(mip_supports))],
    "nash_runtime":[nruntime for i in range(len(mip_supports))],
    "mip_value":mip_us,
    "mip_support":mip_supports,
    "mip_runtime":mip_runtimes,
})
df

,num_timesteps,num_attackers,num_defenders,num_clusters,dims,defense_time_threshold,force_return,num_defender_actions,nash_value,nash_support,nash_runtime,mip_value,mip_support,mip_runtime
0,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.889447,1,33.652055
1,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.597020,2,362.620123
2,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.507538,3,62.383189
3,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.478263,4,117.299998
4,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.471666,5,127.923895
5,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.465396,6,141.439785
6,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.463238,7,20.655978


In [53]:
df.to_csv("GSG_NFG_T9_SPARSITY.csv")

## SF

In [286]:
df = pd.read_csv("lobeke.csv")
df.dropna(inplace=True)

lat_min, lon_min = 2.0530, 15.8790
lat_max, lon_max = 2.2837, 16.2038

coordinate_rectangle = [lat_min, lat_max, lon_min, lon_max]

schedule_form_kwargs = {
    "schedule_form": True,
    "simple": False,
    "attacker_penalty_factor": 5,
    "defender_penalty_factor": 5,
}

general_sum_kwargs = {
    "general_sum": False,
    "attacker_feature_value":  42, 
    "defender_feature_value": 69, 
    "defender_step_cost": 32.5, 
}

boulou_camp = (2.2,15.9)
# lobeke_camp = (2.25,15.75)
kabo_djembe = (2.0532352380408088, 16.085709866529694)
bomassa = (2.2037280296158355, 16.187056364164913)
inner_post = (2.2,15.98)

In [287]:
gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=8, num_columns=8)
gsg.generate(num_attackers=1, 
             num_defenders=2, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post),(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=8, 
             generate_utility_matrix=True, 
             defense_time_threshold=2, 
             generate_actions=False, 
             force_return=True, 
             general_sum=False, 
             **schedule_form_kwargs)

In [288]:
gsg.schedule_form_dict["defender_utility_matrix"].shape

(361, 10)

In [289]:
start = time.time()
nD_a, nD_d, nu = nash(gsg.schedule_form_dict["defender_utility_matrix"])
end = time.time()
nruntime = end-start
nsupport = sum([1 for p in nD_d if p!=0])
nsupport,nu

(6, -0.27729882609690903)

In [290]:
mip_us = []
mip_supports = []
mip_runtimes = []

for i in range(1,10):
    start = time.time()
    print(f"starting i={i} at time {start}")
    mu, mD_d = mip(gsg.schedule_form_dict["defender_utility_matrix"],i)
    end = time.time()
    print(f"finished i={i} in {end-start} seconds with u={mu}")
    msupport = sum([1 for p in mD_d if p!=0])
    mip_us.append(mu)
    mip_supports.append(i)
    mip_runtimes.append(end-start)
    if abs(mu-nu) <= 1e-12:
        break

starting i=1 at time 1745986556.0693774
finished i=1 in 0.09237241744995117 seconds with u=-0.5226130653266332
starting i=2 at time 1745986556.1617498
finished i=2 in 0.24329137802124023 seconds with u=-0.3366140189826578
starting i=3 at time 1745986556.4050412
finished i=3 in 0.13461565971374512 seconds with u=-0.28245611317941893
starting i=4 at time 1745986556.540661
finished i=4 in 0.3160886764526367 seconds with u=-0.28007355003955064
starting i=5 at time 1745986556.8567498
finished i=5 in 0.07416820526123047 seconds with u=-0.2772988260969091


In [291]:
df = pd.DataFrame({
    "num_timesteps":[8 for i in range(len(mip_supports))],
    "num_attackers":[1 for i in range(len(mip_supports))],
    "num_defenders":[2 for i in range(len(mip_supports))],
    "num_clusters":[10 for i in range(len(mip_supports))],
    "dims":[10 for i in range(len(mip_supports))],
    "defense_time_threshold":[2 for i in range(len(mip_supports))],
    "force_return":[True for i in range(len(mip_supports))],
    "num_defender_actions": [gsg.schedule_form_dict["defender_utility_matrix"].shape[0] for i in range(len(mip_supports))],
    "nash_value":[nu for i in range(len(mip_supports))],
    "nash_support":[nsupport for i in range(len(mip_supports))],
    "nash_runtime":[nruntime for i in range(len(mip_supports))],
    "mip_value":mip_us,
    "mip_support":mip_supports,
    "mip_runtime":mip_runtimes,
})
df

,num_timesteps,num_attackers,num_defenders,num_clusters,dims,defense_time_threshold,force_return,num_defender_actions,nash_value,nash_support,nash_runtime,mip_value,mip_support,mip_runtime
0,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.522613,1,0.092372
1,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.336614,2,0.243291
2,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.282456,3,0.134616
3,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.280074,4,0.316089
4,8,1,2,10,10,2,True,361,-0.277299,6,0.061458,-0.277299,5,0.074168


In [292]:
df.to_csv("GSG_SF_T8_SPARSITY.csv")

In [48]:
gsg = GreenSecurityGame(df, coordinate_rectangle, "centroid", num_clusters=10, num_rows=7, num_columns=7)
gsg.generate(num_attackers=2, 
             num_defenders=1, 
             home_base_assignments=[(kabo_djembe, bomassa, inner_post)], 
             num_timesteps=9, 
             generate_utility_matrix=True, 
             defense_time_threshold=1, 
             generate_actions=False, 
             force_return=False, 
             general_sum=False, 
             **schedule_form_kwargs)

In [49]:
gsg.defender_actions.shape

(41479, 9, 1)

In [50]:
start = time.time()
nD_a, nD_d, nu = nash(gsg.utility_matrix)
end = time.time()
nruntime = end-start
nsupport = sum([1 for p in nD_d if p!=0])
nsupport,nu

(8, -0.46323812842864565)

In [51]:
mip_us = []
mip_supports = []
mip_runtimes = []

for i in range(1,10):
    start = time.time()
    print(f"starting i={i} at time {start}")
    mu, mD_d = mip(gsg.utility_matrix,i)
    end = time.time()
    print(f"finished i={i} in {end-start} seconds with u={mu}")
    msupport = sum([1 for p in mD_d if p!=0])
    mip_us.append(mu)
    mip_supports.append(i)
    mip_runtimes.append(end-start)
    if abs(mu-nu) <= 1e-12:
        break

starting i=1 at time 1745981272.9549503
finished i=1 in 33.65205478668213 seconds with u=-0.8894472361809045
starting i=2 at time 1745981306.6102126
finished i=2 in 362.6201231479645 seconds with u=-0.5970202788449397
starting i=3 at time 1745981669.2332556
finished i=3 in 62.38318920135498 seconds with u=-0.5075376884422111
starting i=4 at time 1745981731.6164448
finished i=4 in 117.29999780654907 seconds with u=-0.4782634344465709
starting i=5 at time 1745981848.9164426
finished i=5 in 127.92389488220215 seconds with u=-0.4716662632569949
starting i=6 at time 1745981976.8419363
finished i=6 in 141.43978476524353 seconds with u=-0.4653956597949693
starting i=7 at time 1745982118.2914765
finished i=7 in 20.655978202819824 seconds with u=-0.46323812842864576


In [52]:
df = pd.DataFrame({
    "num_timesteps":[9 for i in range(len(mip_supports))],
    "num_attackers":[2 for i in range(len(mip_supports))],
    "num_defenders":[1 for i in range(len(mip_supports))],
    "num_clusters":[10 for i in range(len(mip_supports))],
    "dims":[7 for i in range(len(mip_supports))],
    "defense_time_threshold":[1 for i in range(len(mip_supports))],
    "force_return":[False for i in range(len(mip_supports))],
    "num_defender_actions": [len(gsg.defender_actions) for i in range(len(mip_supports))],
    "nash_value":[nu for i in range(len(mip_supports))],
    "nash_support":[nsupport for i in range(len(mip_supports))],
    "nash_runtime":[nruntime for i in range(len(mip_supports))],
    "mip_value":mip_us,
    "mip_support":mip_supports,
    "mip_runtime":mip_runtimes,
})
df

,num_timesteps,num_attackers,num_defenders,num_clusters,dims,defense_time_threshold,force_return,num_defender_actions,nash_value,nash_support,nash_runtime,mip_value,mip_support,mip_runtime
0,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.889447,1,33.652055
1,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.597020,2,362.620123
2,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.507538,3,62.383189
3,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.478263,4,117.299998
4,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.471666,5,127.923895
5,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.465396,6,141.439785
6,9,2,1,10,7,1,False,41479,-0.463238,8,18.287839,-0.463238,7,20.655978


In [53]:
df.to_csv("GSG_NFG_T9_SPARSITY.csv")